In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cac40-stocks-dataset/CAC40_stocks_2021_2023.csv
/kaggle/input/cac40-stocks-dataset/preprocessed_CAC40.csv
/kaggle/input/cac40-stocks-dataset/CAC40_stocks_2010_2021.csv


In [2]:
# Load csv
df = pd.read_csv('/kaggle/input/cac40-stocks-dataset/preprocessed_CAC40.csv', parse_dates=['Date'])

In [3]:
df.head()

,Unnamed: 0,Name,Date,Open,Closing_Price,Daily_High,Daily_Low,Volume
0,0,Accor,2020-04-03,22.99,23.40,23.40,22.99,67
1,1,Accor,2020-04-02,23.91,22.99,23.91,22.99,250
2,2,Accor,2020-04-01,24.10,23.83,24.10,23.83,37
3,3,Accor,2020-03-31,25.04,25.00,25.24,24.99,336
4,4,Accor,2020-03-30,26.50,25.02,26.50,24.99,415


In [4]:
# Drop unnecessary columns
df1 = df.drop(['Unnamed: 0', 'Name'], axis=1)
df1.head()

,Date,Open,Closing_Price,Daily_High,Daily_Low,Volume
0,2020-04-03,22.99,23.40,23.40,22.99,67
1,2020-04-02,23.91,22.99,23.91,22.99,250
2,2020-04-01,24.10,23.83,24.10,23.83,37
3,2020-03-31,25.04,25.00,25.24,24.99,336
4,2020-03-30,26.50,25.02,26.50,24.99,415


In [5]:
# Convert date to datetime
df1['Date'] = pd.to_datetime(df1['Date'])

In [6]:
# Set date as index
df1 = df1.set_index('Date')

In [7]:
df1.isna().sum()

Open               230
Closing_Price        2
Daily_High         204
Daily_Low          204
Volume           20453
dtype: int64

In [8]:
from sklearn.impute import SimpleImputer

# Impute missing values with mean
imputer = SimpleImputer(strategy='mean')
df1[['Open', 'Closing_Price', 'Daily_High', 'Daily_Low']] = imputer.fit_transform(df1[['Open', 'Closing_Price', 'Daily_High', 'Daily_Low']])

In [9]:
import numpy as np

df1['Volume'] = df1['Volume'].apply(lambda x: np.nan if not isinstance(x, (int, float)) else x)

In [10]:
imputer = SimpleImputer(strategy='most_frequent')
df1['Volume'] = imputer.fit_transform(df1['Volume'].values.reshape(-1, 1))

In [11]:
from sklearn.preprocessing import MinMaxScaler
# Scale data
scaler = MinMaxScaler()
df1['Closing_Price'] = scaler.fit_transform(df1['Closing_Price'].values.reshape(-1, 1))

In [12]:
# Create features and labels
X = []
y = []
window_size = 30  # Lookback window size

In [13]:
for i in range(window_size, len(df1) - 3):
    X.append(df1['Closing_Price'].values[i - window_size:i])
    y.append(df1['Closing_Price'].values[i:i + 3])

In [14]:
import numpy as np
X = np.array(X)
y = np.array(y)

In [15]:
# Split data into train and test sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [16]:
# Reshape data for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
# Build and train LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(window_size, 1)))
model.add(Dense(3))  # Output shape: (None, 3) for 3-day prediction
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

2024-03-16 05:42:58.721606: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-16 05:42:58.721781: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-16 05:42:58.902895: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
2441/2441 ━━━━━━━━━━━━━━━━━━━━ 44s 17ms/step - loss: 7.6860e-04 - val_loss: 3.3574e-05
Epoch 2/100
2441/2441 ━━━━━━━━━━━━━━━━━━━━ 81s 17ms/step - loss: 5.9475e-05 - val_loss: 1.2848e-05
Epoch 3/100
2441/2441 ━━━━━━━━━━━━━━━━━━━━ 42s 17ms/step - loss: 5.4222e-05 - val_loss: 1.3299e-05
Epoch 4/100
2441/2441 ━━━━━━━━━━━━━━━━━━━━ 42s 17ms/step - loss: 6.5810e-05 - val_loss: 1.0810e-05
Epoch 5/100
2441/2441 ━━━━━━━━━━━━━━━━━━━━ 81s 17ms/step - loss: 2.9672e-05 - val_loss: 1.0076e-05
Epoch 6/100
2441/2441 ━━━━━━━━━━━━━━━━━━━━ 82s 17ms/step - loss: 3.9719e-05 - val_loss: 1.0425e-05
Epoch 7/100
2441/2441 ━━━━━━━━━━━━━━━━━━━━ 41s 17ms/step - loss: 5.5047e-05 - val_loss: 1.8595e-05
Epoch 8/100
2441/2441 ━━━━━━━━━━━━━━━━━━━━ 41s 17ms/step - loss: 5.2539e-05 - val_loss: 1.2976e-05
Epoch 9/100
2441/2441 ━━━━━━━━━━━━━━━━━━━━ 82s 17ms/step - loss: 3.8914e-05 - val_loss: 1.2017e-05
Epoch 10/100
2441/2441 ━━━━━━━━━━━━━━━━━━━━ 41s 17ms/step - loss: 3.0713e-05 - val_loss: 1.9084e-05
Epoch 11/

In [18]:
# Make predictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

611/611 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step


In [19]:
predictions

array([[102.33142 , 102.12079 , 102.12815 ],
       [101.38655 , 101.052025, 100.98773 ],
       [ 99.527336,  99.04941 ,  98.92146 ],
       ...,
       [ 16.550594,  16.413258,  16.736498],
       [ 16.538742,  16.398031,  16.719738],
       [ 16.553204,  16.414171,  16.737612]], dtype=float32)

In [20]:
from sklearn.metrics import mean_squared_error
# Get the actual values from y_test
y_test = scaler.inverse_transform(y_test)

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f"RMSE: {rmse}")

RMSE: 2.2685208578152585


In [21]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

mae = mean_absolute_error(y_test, predictions)
mape = mean_absolute_percentage_error(y_test, predictions)
print(f"MAE: {mae}")
print(f"MAPE: {mape}")

MAE: 0.9666040403396122
MAPE: 0.02474898972822684
